In [1]:
import qiskit

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

qiskit_runtime_service.__init__:WARNING:2025-11-15 08:18:55,599: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# Load IBM account
service = QiskitRuntimeService()

# Select backend (hardware)
backend = service.backend("ibm_fez")

# Make circuit
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)


qc_t = transpile(qc, backend)

# Run on hardware through SamplerV2
sampler = SamplerV2(backend)
job = sampler.run([qc_t])   # submit job

import time
print("Job submitted. Status:", job.status())
while not job.done():
    print("Job status:", job.status())
    time.sleep(10)  # Waits 10 seconds before next check

result = job.result()
print("Job finished!", job.status())
print(result)




qiskit_runtime_service.__init__:WARNING:2025-11-15 08:19:05,218: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-15 08:19:05,223: Using instance: open-instance, plan: open


Job submitted. Status: QUEUED
Job status: RUNNING
Job finished! DONE
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-11-15 02:49:12', stop='2025-11-15 02:49:14', size=4096>)])}, 'version': 2})


In [4]:
# eve_qiskit.py
from flask import Flask, request, jsonify
import numpy as np, time, requests, random
# We'll try to use qiskit primitives; if not available we fall back to analytic simulator
try:
    from qiskit import QuantumCircuit, transpile
    from qiskit.primitives import Sampler
    _HAS_SAMPLER = True
except Exception:
    # sampler not available (common on Windows without Aer); we will use physics-faithful fallback
    from qiskit import QuantumCircuit  # only for clarity (we won't run circuits)
    _HAS_SAMPLER = False

app = Flask(__name__)
ALICE_IP = "192.168.1.10"
BOB_HOST = f"http://192.168.1.10:5002"

INTERCEPT_PROB = 0.25  # chance Eve intercepts each qubit

if _HAS_SAMPLER:
    sampler = Sampler()  # Qiskit primitive (2.x)

def build_prep_circuit(bit, basis, meas_basis=None):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)
    if basis == 1:
        qc.h(0)
    if meas_basis is not None and meas_basis == 1:
        qc.h(0)
    qc.measure(0, 0)
    return qc

def measure_physical(bit, prep_basis, meas_basis):
    """
    Analytic (exact) single-qubit measurement behaviour:
    - if prep_basis == meas_basis -> measured == bit (deterministic)
    - else -> random 0/1 (50/50)
    """
    if prep_basis == meas_basis:
        return int(bit)
    else:
        return int(random.getrandbits(1))

def measure_with_sampler(bit, prep_basis, meas_basis):
    qc = build_prep_circuit(bit, prep_basis, meas_basis)
    tqc = transpile(qc, sampler.target) if hasattr(sampler, "target") else qc
    job = sampler.run([qc])
    result = job.result()
    # primitive SamplerResult API varies; try common extraction methods
    try:
        # new API: result.quasi_dists or result.__dict__
        # we'll try result.quasi_dists[0].binary_probabilities() or result.binary_probabilities(0)
        # safe fallback: get counts-like dict
        qdict = result.get_counts(0)  # if this exists
        measured = int(list(qdict.keys())[0])
    except Exception:
        try:
            # try other method:
            measured = int(list(result.quasi_dists[0].to_dict().keys())[0])
        except Exception:
            # last fallback - random
            measured = int(random.getrandbits(1))
    return measured

def measure_on_backend(bit, prep_basis, meas_basis):
    if _HAS_SAMPLER:
        return measure_with_sampler(bit, prep_basis, meas_basis)
    else:
        return measure_physical(bit, prep_basis, meas_basis)

@app.route("/receive_from_alice", methods=["POST"])
def receive_from_alice():
    data = request.json
    preps = data.get("preps", [])
    resend_list = []
    ping_list = []
    for item in preps:
        idx = item["idx"]
        a_bit = int(item["bit"])
        a_basis = int(item["basis"])
        do_intercept = (np.random.rand() < INTERCEPT_PROB)
        if do_intercept:
            eve_basis = int(np.random.randint(2))
            eve_result = measure_on_backend(a_bit, a_basis, eve_basis)
            resend_list.append({"idx": idx, "bit": int(eve_result), "basis": int(eve_basis), "intercepted": True})
            if eve_basis != a_basis:
                ping_list.append(idx)
        else:
            resend_list.append({"idx": idx, "bit": a_bit, "basis": a_basis, "intercepted": False})
    try:
        r = requests.post(f"{BOB_HOST}/receive_from_eve", json={"preps": resend_list}, timeout=30)
        return jsonify({"status": "relayed_to_bob", "bob_response": r.text, "pings": ping_list})
    except Exception as e:
        print("Eve -> Bob failed:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    print("Eve server running on port 5001  (sampler available: {})".format(_HAS_SAMPLER))
    app.run(host="0.0.0.0", port=5001)


Eve server running on port 5001  (sampler available: False)
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.107:5001
Press CTRL+C to quit
